In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
train_data.head()

In [ ]:
train_data.describe()

In [ ]:
trainy=train_data['target']
trainx=train_data.drop(["id","target"],axis=1)
trainy=np.array(pd.get_dummies(trainy))
trainx.describe()

In [ ]:
# from sklearn.datasets import load_iris
# from sklearn import preprocessing
# trainx = preprocessing.normalize(trainx)
maxx=trainx.max(axis=0)
minx=trainx.min(axis=0)
trainx=(trainx-minx)/(maxx-minx)
trainx=np.c_[np.ones((trainx.shape[0],1)),trainx]

In [ ]:
def softmax(z):
    return np.exp(z) / np.sum(np.exp(z),axis=1).reshape(z.shape[0],1)

In [ ]:
def cost(x,y,theta):
    z=np.dot(x,theta)
    a=softmax(z)
    return (-1/x.shape[0])*np.sum(y*np.log(a))

In [ ]:
def grad(x,y,theta,alpha,epochs):
    m=x.shape[0]
    costs=[]
    for i in range(epochs):
        h=softmax(np.dot(x,theta))
        theta+=(alpha/m)*(np.dot(x.T,y*(1-h)))
        costs.append(cost(x,y,theta))
    return theta,costs

In [ ]:
theta=np.random.normal(scale=1/(trainx.shape[1]**0.5),size=(trainx.shape[1],trainy.shape[1]))
theta,costs=grad(trainx,trainy,theta,0.01,1000)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title('Cost reduction over time')
plt.show()

In [ ]:
test_data=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
test_data.head()

In [ ]:
test_data.describe()

In [ ]:
testx=test_data.drop(["id"],axis=1)
testx.describe()

In [ ]:
testx=(testx-minx)/(maxx-minx)
testx=np.c_[np.ones((testx.shape[0],1)),testx]

In [ ]:
testz=np.dot(testx,theta)
testh=softmax(testz)
df=pd.DataFrame(testh)
df

In [ ]:
print(testh.shape)
print(np.max(testh,axis=1).shape)
out=pd.DataFrame((testh==np.max(testh,axis=1).reshape(testh.shape[0],1)).astype(int))
print(out.sum(axis=0))

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

sub[sub.columns[1:]] = df

sub.to_csv("my_submission.csv",index = False)
sub.head()